<a href="https://colab.research.google.com/github/rainmaker29/ML701_G12/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import cv2
import numpy as np

import zipfile
from PIL import Image
from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

In [3]:
# Reading the data

with zipfile.ZipFile("/content/train_.zip", "r") as zip_data:
    content_list = zip_data.namelist()
    labels = []
    for folder in ['train_/GALAXY/', 'train_/STAR/', 'train_/QSO/']:
        content_list.remove(folder)
    for _, name_file in enumerate(content_list):
        img_bytes = zip_data.open(name_file)
        img_data = Image.open(img_bytes)
        image_as_array = np.array(img_data, np.uint8)
        if _ == 0:
            data = image_as_array[np.newaxis, :]
        else:
            data = np.concatenate((data, image_as_array[np.newaxis, :]))
        if 'GALAXY' in name_file:
            labels.append('GALAXY')
        elif 'QSO' in name_file:
            labels.append('QSO')
        elif 'STAR' in name_file:
            labels.append('STAR')

# Shuffling the data

labels = np.array(labels)

shuffler = np.random.RandomState(seed=10).permutation(data.shape[0])

y_not_encoded = labels[shuffler]
y = np.unique(y_not_encoded, return_inverse=True)[1]

x = data[shuffler]
x = x.astype('float32')
x /= 255

# Saving the data
import h5py

with h5py.File('images_matrix.h5', 'w') as hf:
    hf.create_dataset("images_matrix",  data=x)

with h5py.File('labels.h5', 'w') as hf:
    hf.create_dataset("labels",  data=y)

In [4]:
# Reading the data

#with h5py.File('images_matrix.h5', 'r') as hf:
#    testing = hf['images_matrix'][:]

# Splitting the data on train and test sets

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=10, test_size = 0.2)

# Encoding the labels

n_classes = 3
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

In [13]:
# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(128, 128, 3)))

# convolutional layer
model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(3, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
63/63 [==============================] - 35s 372ms/step - loss: 0.4372 - accuracy: 0.8316 - val_loss: 0.3355 - val_accuracy: 0.8780
Epoch 2/10
63/63 [==============================] - 20s 312ms/step - loss: 0.2236 - accuracy: 0.9169 - val_loss: 0.2170 - val_accuracy: 0.9260
Epoch 3/10
63/63 [==============================] - 20s 312ms/step - loss: 0.1819 - accuracy: 0.9396 - val_loss: 0.2059 - val_accuracy: 0.9375
Epoch 4/10
63/63 [==============================] - 20s 315ms/step - loss: 0.1446 - accuracy: 0.9535 - val_loss: 0.2429 - val_accuracy: 0.9180
Epoch 5/10
63/63 [==============================] - 20s 319ms/step - loss: 0.1431 - accuracy: 0.9523 - val_loss: 0.2090 - val_accuracy: 0.9395
Epoch 6/10
63/63 [==============================] - 20s 321ms/step - loss: 0.1295 - accuracy: 0.9561 - val_loss: 0.3116 - val_accuracy: 0.8895
Epoch 7/10
63/63 [==============================] - 20s 324ms/step - loss: 0.1123 - accuracy: 0.9628 - val_loss: 0.2209 - val_accuracy: 0.9425

In [15]:
predictions = model.predict(x_test)

63/63 [==============================] - 2s 24ms/step


array([0.9984743 , 0.9997489 , 0.9886453 , ..., 0.9878862 , 0.99997985,
       0.99957675], dtype=float32)

In [20]:
np.where(predictions[0] == np.max(predictions[0], axis=1), 1, 0)

AxisError: ignored

In [19]:
predictions.shape

(2000, 3)

In [ ]:
for n in range(predictions.shape[0]):
    